  # Integrantes del equipo:
- Juan Camilo Carvajal Sierra - 1010124738 - Bioingeniería
- Valeria Giraldo Agudelo - 1007577551 - Bioingeniería
- Mauro Andres Alegria Cano - 1002972153 - Bioingeniería

##Importación de librerias

In [6]:
import numpy as np
import pandas as pd
import os
import re
import unicodedata
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

##Descarga de archivos desde Kaggle

In [7]:
# os.environ["KAGGLE_CONFIG_DIR"] = "."
# !kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
# !unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
df = pd.read_csv("train.csv")
df.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


##Creación de la funcion para limpiar el dataframe

In [3]:
def one_hot_encode_column(df, column_name): #Función para crear One Hot
  df = df.fillna(method='ffill') #Este método rellena los valores faltantes en la columna, utilizando el último valor válido hacia adelante para rellenar los valores faltantes.
  unique_values = df[column_name].unique() # Se obtenien valores únicos en la columna
  for value in unique_values: # Se crean columnas one-hot para cada valor único
      df[column_name + '_' + str(value)] = (df[column_name] == value).astype(int)
  del df[column_name]
  return df

def obtener_valor_medio(rango): #Función para convertir valores string a int, luego sacar el valor medio de la columna
  if isinstance(rango, str):  #Verifica si el argumento dado es una cadena de texto(string)
      numeros = [int(num) for num in rango.split() if num.isdigit()] #Divide la cadena rango en palabras y crea una lista de números enteros (int) a partir de cada palabra que sea un número filtrando con if num.isdigit() filtra solo las palabras que son números.
      return sum(numeros) / len(numeros) #Devuelve el promedio de los números
  else:
      return float('nan')  #Devuelve un valor nan en caso de que no sea una cadena de texto

def obtener_valor_medio2(rango): #Función para convertir valores string a int, luego sacar el valor medio de la columna considerando los casos especiales
  if isinstance(rango, str): #Verifica si el argumento dado es una cadena de texto(string)
      # Reemplazar datos específicas por los valores correspondientes para los casos especiales
      rango = rango.replace("No pagó matrícula", "0")
      rango = rango.replace("Menos de 500 mil", "0.5")
      rango = rango.replace("Entre 500 mil y menos de 1 millón", "0.75")
      # Utilizar expresiones regulares para encontrar números decimales en la cadena
      numeros = re.findall(r"\d+\.\d+|\d+", rango) #Utiliza expresiones regulares \d+\.\d+|\d+ para buscar patrones de números con o sin decimales
      # Convertir los números a flotantes y calcular la media
      numeros = [float(num) for num in numeros] #Convierte los números encontrados a números flotantes (float).
      if numeros:
          return sum(numeros) / len(numeros) #Si en la lista hay numeros, devuelve el promedio de los números en la lista
      else:
          return float('nan') #Devuelve un valor nan en caso de que no sea un número
  else:
      return float('nan') #Devuelve un valor nan en caso de que no sea una cadena de texto

def normalize_text(text): #Funcion para eliminar tildes y caracteres especiales
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    text = text.lower().strip()  # Convertir a minúsculas y eliminar espacios en blanco
    text = text.replace('-', ' ').replace('  ', ' ').replace(".","")  # Reemplazar guiones por espacios y eliminar espacios dobles
    return text

def Limpieza(df): #Funcion que encapsula las otras funciones para hacer una limpieza o preprocesado completo al Dataframe
    df.drop(columns=['FAMI_TIENEINTERNET.1', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO'], inplace=True) #Se eliminan columnas que se consideran no son de utilidad

    #Se aplican las funciones a las columnas
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].apply(obtener_valor_medio)
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna(df['ESTU_HORASSEMANATRABAJA'].median())

    df["ESTU_VALORMATRICULAUNIVERSIDAD"] = df["ESTU_VALORMATRICULAUNIVERSIDAD"].apply(obtener_valor_medio2)
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(df['ESTU_VALORMATRICULAUNIVERSIDAD'].median())

    df["FAMI_ESTRATOVIVIENDA"] = df["FAMI_ESTRATOVIVIENDA"].apply(obtener_valor_medio2)
    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna(df['FAMI_ESTRATOVIVIENDA'].median())

    df = one_hot_encode_column(df, 'FAMI_TIENEINTERNET')
    df = one_hot_encode_column(df, 'FAMI_EDUCACIONPADRE')
    df = one_hot_encode_column(df, 'FAMI_TIENECOMPUTADOR')
    df = one_hot_encode_column(df, 'FAMI_EDUCACIONMADRE')
    df = one_hot_encode_column(df, 'ESTU_PRGM_DEPARTAMENTO')

    df['ESTU_PRIVADO_LIBERTAD'] = df['ESTU_PRIVADO_LIBERTAD'].replace({'N': 'No', 'S': 'Si'})
    df = one_hot_encode_column(df, 'ESTU_PRIVADO_LIBERTAD')

    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(normalize_text)

    #Se realiza una limpieza manual a la columna de programas para los datos que tienen simbolos dificiles de estandarizar en una función
    replacements = {
        'administraci¿n de empresas': 'administracion de empresas',
        'ingenier¿a en software': 'ingenieria en software',
        'administraci¿n log¿stica': 'administracion logistica',
        'licenciatura en educaci¿n art¿stica': 'licenciatura en educacion artistica',
        'licenciatura en pedagog¿a infantil': 'licenciatura en pedagogia infantil',
        'dise¿¿o de comunicaci¿¿n gr¿¿fica': 'diseno de comunicacion grafica',
        'geolog¿a': 'geologia',
        'ingenier¿a de sistemas': 'ingenieria de sistemas',
        'licenciatura en educaci¿n b¿sica primaria': 'licenciatura en educacion basica primaria',
        'profesional en gastronom¿a': 'profesional en gastronomia',
        'econom¿a': 'economia',
        'comunicaci¿n social y periodismo': 'comunicacion social y periodismo',
        'contaduria p¿blica': 'contaduria publica',
        'licenciatura en educaci¿n con ¿nfasis en ciencias sociales y ambientales': 'licenciatura en educacion con enfasis en ciencias sociales y ambientales',
        'licenciatura m¿sica': 'licenciatura musica',
        'gesti¿n deportiva': 'gestion deportiva',
        'licenciatura en lenguas extranjeras con ¿nfasis en ingl¿s': 'licenciatura en lenguas extranjeras con enfasis en ingles',
        'dise¿o y administraci¿n de negocios de la moda': 'diseno y administracion de negocios de la moda',
        'licenciatura en educaci¿n infantil': 'licenciatura en educacion infantil',
        'comunicaci¿n social': 'comunicacion social',
        'administraci¿n p¿blica': 'administracion publica',
        'qu¿mica farmac¿utica': 'quimica farmaceutica',
        'ingenier¿a el¿ctrica': 'ingenieria electrica',
        'comunicaci¿n visual': 'comunicacion visual',
        'ingenier¿a industrial': 'ingenieria industrial',
        'dise¿o crossmedia': 'diseno crossmedia',
        'licenciatura en filosof¿a y humanidades': 'licenciatura en filosofia y humanidades',
        'licenciatura en educaci¿n f¿sica recreaci¿n y deportes': 'licenciatura en educacion fisica recreacion y deportes',
        'licenciatura en artes esc¿nicas': 'licenciatura en artes escenicas',
        'licenciatura en matem¿ticas': 'licenciatura en matematicas'
    }

    for original, replacement in replacements.items():
        df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].str.replace(original, replacement)

    df = one_hot_encode_column(df, 'ESTU_PRGM_ACADEMICO')

    return df

##Se importan los archivos de train y test de kaggle que se usaran

In [4]:
dtr = pd.read_csv("train.csv")
dts = pd.read_csv("test.csv")
lentr = len(dtr)
dtr.shape, dts.shape

((692500, 21), (296786, 20))

##Se realiza la divición de los datos que se usaran para entrenar y para probar

In [5]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = Limpieza(all_data)

Xtr, ytr = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xts      = all_data.iloc[lentr:].values

print (Xtr.shape, ytr.shape)
print (Xts.shape)

C:\Users\juanc\AppData\Local\Temp\ipykernel_9004\2829738752.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill') #Este método rellena los valores faltantes en la columna, utilizando el último valor válido hacia adelante para rellenar los valores faltantes.
C:\Users\juanc\AppData\Local\Temp\ipykernel_9004\2829738752.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill') #Este método rellena los valores faltantes en la columna, utilizando el último valor válido hacia adelante para rellenar los valores faltantes.
C:\Users\juanc\AppData\Local\Temp\ipykernel_9004\2829738752.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill') #Este m

KeyboardInterrupt: 

In [ ]:
n=int(len(Xtr)*0.7)
n

484749

In [ ]:
idxs=np.random.permutation(len(Xtr))
idxs_trm=idxs[:n]
idxs_tsm=idxs[n:]

xtrm=Xtr[idxs_trm]
ytrm=ytr[idxs_trm]

xtsm=Xtr[idxs_tsm]
ytsm=ytr[idxs_tsm]

xtrm.shape, ytrm.shape, xtsm.shape,ytsm.shape

((484749, 810), (484749,), (207751, 810), (207751,))

##Se inicia el codigo para entrenamiento

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)

In [ ]:
gbc.fit(xtrm, ytrm) #entrenar el modelo

,loss,'log_loss'
,learning_rate,1.0
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,1
,min_impurity_decrease,0.0
,init,None


In [ ]:
y_pred = gbc.predict(Xts) #se hacen las predicciones del modelo
y_pred.shape

(296786,)

In [ ]:
preds_tsm=gbc.predict(xtsm)
preds_trm=gbc.predict(xtrm)
acc_tsm=(preds_tsm==ytsm).mean()
acc_trm=(preds_trm==ytrm).mean()
print(f"precision en entranamiento(train_m) {acc_trm:.3f}")
print(f"precision en prueba(test_m) {acc_tsm:.3f}")

precision en entranamiento(train_m) 0.418
precision en prueba(test_m) 0.416


In [ ]:
preds_tsk=gbc.predict(Xts)

In [ ]:
Prediccionesdf=dts[["ID"]].copy()
Prediccionesdf["RENDIMIENTO_GLOBAL"]=y_pred
Prediccionesdf.to_csv('prediccionesGBC.csv', index=False)